In [1]:
import pandas, numpy as np,re
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
import datetime, tqdm
pars =HTMLParser()
def findInXML(s, et): 
    r =et.find('.//row[@NombreCampo="%s"]' % s)
    if r is not None:
        return r.get('ValorCampo')
    return None
def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(r.toprettyxml())
    
#from datetime import datetime as dt
#import locale
#locale.setlocale(locale.LC_ALL, 'es_es')
#spanishDateToPandas = lambda s: pandas.to_datetime(dt.strptime(s, '%d %b %Y %H:%M:%S'))

# Data reading and basic cleaning

- Find candidates of repeated pateints
- Find the used codes for diagnosis and procedures

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)

#str2Date = lambda s:datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f') if isinstance(s, str) \
#           else datetime.datetime(1, 1, 1)
registros.FechaInicioAtencion = registros.FechaInicioAtencion.map(pandas.to_datetime) #registros.FechaInicioAtencion.map(str2Date)
procedimientos.FechaDescripcion  = procedimientos.FechaDescripcion.map(pandas.to_datetime)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,27,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
procedureIDFound = set()
abortions = []
def getProcFromXMLString(s):
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)
    return s[0]

for k, p in procedimientos.iterrows():
    et = xml.etree.ElementTree.fromstring(p.XmlDescripcion)
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', p.XmlDescripcion)
    for ss in s:
        procedureIDFound.add(ss)
    if 'H3094' in p.XmlDescripcion:
        abortions.append(k)
procedureIDFound = list(sorted(procedureIDFound))

In [4]:
procedimientosDesc.loc[procedureIDFound].to_excel('procedimientosFiltered.xlsx')
procedimientosDesc.loc[procedureIDFound].to_csv('procedimientosFiltered.csv')

In [5]:
unique = registros.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
for u in unique:
    if u.upper() not in diagnosis.index:
        print(u)

0800


In [6]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

# Select the cases by procedure

- Based on the procedure, select the history register associated.
- TODO: Check that diagnosis and procedure are compatible

In [7]:
registersByPatient = registros.groupby('NumeroHistoria')

In [9]:
interventionToCase = {}
caseToProcedure = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcion = int(et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo'))
        interventionToCase[idDescripcion] = (r.Caso, r.NumeroHistoria, i)
        caseToProcedure[idDescripcion] = getProcFromXMLString(procedimientos.loc[idDescripcion].XmlDescripcion)

In [10]:
errors = []
for i, (_, k, k2) in interventionToCase.items():
    if procedimientos.loc[i].IdPaciente != k:
        print('ERROR: Patient name does not coincides', i, k2)
        errors.append(i)
for i in errors:
    del interventionToCase[i]

ERROR: Patient name does not coincides 61066 1356698


# Parse information from cases
Now we need to parse the information from the cases

In [11]:
def getStateDataFromCase(case):
    """
    Get some information from the case.
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [12]:
floatParse = '[0-9]*[\.,]?[0-9]+'
compareGPCA = lambda s1, s2: all([i == j or not i or not j for i, j in zip(s1, s2)])
allEqual = lambda s: all([compareGPCA(i, j) for i, j in  itertools.combinations(s, r = 2)])
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGPAC = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGPAC += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGPAC += '|'
patternGPAC += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGPAC += '|' # C and A can be missing, or in other order
patternGPAC += ')'
searchGPAC = re.compile(patternGPAC, flags=0)

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
noRecuerda = ['NO', '\?']
separadorFecha = '(?:[\.\\/-]|DE|DEL|\s)'
date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

searchFUM = re.compile('FUM'+ sep + '(?::|.)?'+ sep +'(' +  date + '|%s)' % ('|'.join(noRecuerda)), flags = re.IGNORECASE)
errors = []
def parseGPCA_and_fum(s):
    """
    Gets the GPCA and FUM from the Triage or epicrisis.
    
    NOTE: some of the cases are incorrect, double check
    TODO: actually, registro del recien nacido has it as a field.
    """
    r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',ET.fromstring(s).findall('row')))
    if len(r) == 0:
         return {}
    text = pars.unescape(r[0].attrib['ValorCampo'])
    
    allowedStarts = ['7-&gt;', '-', '- ANTECEDENTES']
    line = re.findall('^%s(?:%s)?%s' % (sep, '|'.join(allowedStarts), sep) + 'G' + sep + '[0-9]+.*$', text, re.M)
    if len(line) != 1:
        #If it doesn't find it, search brutally
        search = searchGPAC.findall(s)
        searchCleaned = [(s[0], s[1], s[2] + s[-1], s[3] + s[-2]) for s in search]
        if search and allEqual(search):
            GPCA_OK = True
            parsedGPCA = search[0]
        else:
            GPCA_OK = False
            parsedGPCA = []

    else:
        f = lambda s: s[0] if s else ''
        G = f(re.findall('G' + sep + '([0-9]+)', line[0]))
        A = f(re.findall('A' + sep + '([0-9]+)', line[0]))
        C = f(re.findall('C' + sep + '([0-9]+)', line[0]))
        P = f(re.findall('P' + sep + '([0-9]+)', line[0])) 
        parsedGPCA = [G, P, A, C]
        GPCA_OK = True
    #Prob athere is a better way...
    parsedFUM = searchFUM.findall(s)
    return {'fum' : len(parsedFUM) > 0,
            'fum_OK' : len(parsedFUM) ,
            'GPCA_OK' : GPCA_OK,
            'fum_Data' : parsedFUM,
            'parsedGPCA' : parsedGPCA}

In [13]:
from IPython.display import display

def getBackgroundRegister(casoID, registrosByCase):
    """
    Gets a register that includes the personal background in order Epicrisis / Triage / Ingreso a urgencias
    """
    #
    epicrisis = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Epicrisis', case = False) == True)]
    if epicrisis is not None and len(epicrisis):
        return epicrisis, 'epi'
    
    triage = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Triage', case = False) == True)]
    if triage is not None and len(triage):
        return triage, 'tri'

    urgencias = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Ingreso de Urgencias', case = False) == True)]
    if urgencias is not None:
        return urgencias, 'ur'


In [14]:
errorsBackgroundRegister = []
backgroundByCase = {}
dataGPAC_fum = {}
typeCases = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    if case is False:
        continue
        
    backgroundByCase[case], typeCases[case] = getBackgroundRegister(case, registersByPatient.get_group(pId))
    if len(backgroundByCase[case]) == 0:
        errorsBackgroundRegister.append(case)
    else: 
        dataGPAC_fum[case] = parseGPCA_and_fum(backgroundByCase[case].iloc[0].RegistroXML)
        c = getProcFromXMLString(procedimientos.loc[k].XmlDescripcion)
        #dataGPAC_fum[case] ['ProcCode'] = c
        #dataGPAC_fum[case] ['ProcDesc'] = procedimientosDesc.loc[c].DESCRIP
    
print('No background', len(errorsBackgroundRegister))

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.



No background 5


In [15]:
def getAllProceduresAndMedicamentsFromCase(registros, Caso):
    procedimientos = []
    medicamentos = []
    for r,v in registros[registros.Caso == Caso].iterrows():
        et = ET.fromstring(v.RegistroXML)
        p = findInXML('ProcedimientosAsociados', et)
        m = findInXML('MedicamentosAdministrado', et)
        if p:
            procedimientos += [p]
        if m:
            medicamentos += [m]
    return procedimientos, medicamentos

medicationByCase = {}
proceduresByCase = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    proceduresByCase[case], medicationByCase[case] = getAllProceduresAndMedicamentsFromCase(registros, case)

In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
patientsWithPeniciline = []
for c, v in medicationByCase.items():
    if 'PENI' in ''.join(v):
        patientsWithPeniciline.append(c)
        

In [18]:
len(patientsWithPeniciline)

96

In [19]:
testSifilis = '906039 - TREPONEMA PALLIDUM, ANTICUERPOS PARA SIFILIS, PRUEBAS RAPIDAS  POR INMUNOCROMATOGRAFIA'
vdrl = '906916' #Prueba no treponemica a sifilis
vih1 = '906249'
vih2 = '906249'

In [20]:
patientsTestedForSifilis = []
patientsTestedForVIH = []
for c, v in proceduresByCase.items():
    v = ''.join(v)
    if vih1 in v:
        patientsTestedForVIH.append(c)  
    if testSifilis in v or vdrl in v:
        patientsTestedForSifilis.append(c)  

In [21]:
len(patientsTestedForSifilis)

2210

In [22]:
casesIdx = list(proceduresByCase.keys())
def printMedication(c):
    print(''.join(medicationByCase[c]))
interact(lambda i: printMedication(casesIdx[i]), i = (0, len(casesIdx)))

interactive(children=(IntSlider(value=1232, description='i', max=2464), Output()), _dom_classes=('widget-inter…

<function __main__.<lambda>(i)>

In [23]:
for c in casos:
    print(c)

FechaHora
Reingreso
Paciente
Acompañante
DireccionAcompanante
TelefonoAcompanante
Particular
Diagnostico
Prestador
Auxiliar1
Auxiliar2
ViaIngreso
CausaExterna
IdPlanAdministradora
UnidadFuncionalInicial
UnidadFuncionalActual
UnidadFuncionalFinal
FechaSalida
DiagnosticoSalida
EstadoSalida
EstadoCaso
Cama
Usuario
Autorizacion
RemitidaA
CiudadRemitida
FechaRemision
CausaMuerte
FechaHoraMuerte
PrestadorDefuncion
CausaEgreso
Triage
Glasgow
Remitido
FechaRemitido
InstitucionRemitido
MunicipioRemision
Catastrofe
FechaCatastrofe
DireccionCatastrofe
ZonaCatastrofe
MunicipioCatastrofe
TipoEventoCatastrofe
EventoCatastrofico
TipoTratamiento
IdPlanFinal
CodigoIntitucionRemitente
ProfesionaRemitente
CargoProfesionalRemitente
IdTriage
CodigoEntidadaRecibe
ProfesionalRecibe
CargoRecibe
HoraRecibe
FechaRecibido
NombreResponsable
TelefonoResponsable
ParentescoResponsable
DireccionResponsable
CentroAtencion
EstadoAseguramiento
TipoCaso
IdUsuario
EstadoFecha
EstadoUsuario
AltaMedica
NumeroRequisitos
Requ

In [24]:
for c, v in casos.iterrows():
    if v.Reingreso:
        print(c, v.Reingreso, v.EstadoSalida)

AD362553 True Vivo


In [25]:
for c, v in proceduresByCase.items():
    if c not in casos.index:
        print(c)

NN


## Quality check
- See how many have epicrisis

In [61]:
findInXML = lambda  s, et: et.find('.//row[@NombreCampo="%s"]' % s).get('ValorCampo')

def ProcessRecienNacido(text):
    et = ET.fromstring(text)
    findInXML = lambda  s: et.find('.//row[@NombreCampo="%s"]' % s).get('ValorCampo')
    fecha = findInXML('InputText_FechaHoraNacimiento')
    hora = findInXML('ASPxTimeEdit_HoraNacimiento')
    EG = findInXML('EdadGestacapuroo')
    EG2 = findInXML('InputText_EdadGestacDubowitzModificado')
    partoVag = findInXML('TexTarea_PartoVaginal')
    partoC = findInXML('TexTarea_PartoCesaria')
    APGAR = findInXML('InputText_APGAR')
    sexo = findInXML('ASPxComboBox_Sexo')
    vivo = findInXML('InputRadio_VM')
    #Antrhopometrics
    weight = findInXML('InputText_Peso')
    height = findInXML('InputText_Talla')
    cc = findInXML('InputText_CC')
    
    #As a check GAPC
    G = findInXML('InputText_ObtetricosGestaciones')
    A = findInXML('InputText_ObtetricosAbortos')
    P = findInXML('InputText_ObtetricosPartos')
    C = findInXML('InputText_ObtetricosCesarias')
    
    #If exposed to sifilis/VIH, it should be here 
    

In [165]:
len(registros[registros.Asunto == 'Registro del recién nacido'].Caso.unique())

2056

In [170]:
vivos, muertos = 0, 0

for k, p in registros[registros.Asunto == 'Registro del recién nacido'].iterrows():
    k
    et = ET.fromstring(p.RegistroXML)
    if findInXML('InputRadio_VM', et) == 'Muerto':
        muertos += 1
    else:
        vivos += 1
    if 'desgarros' in p.RegistroXML.lower():
        print('found')
        break
print(vivos, muertos, muertos/(vivos + muertos))

found
1251 8 0.006354249404289118


In [ ]:
AD213297 -> caso de nacimiento

In [46]:
for p, k in backgroundByCase.items():
    xmlET = ET.fromstring(k.iloc[0].RegistroXML)
    if 'VDRL: REACTIVO' in xmlET.find('.//row[@NombreCampo="Evoluciones"]').get('ValorCampo').upper():
        prettyPrintXML(k.iloc[0].RegistroXML)
        print(k.Diagnostico, k.CodigoDiagnosticoRelacionado1, k.CodigoDiagnosticoRelacionado2, k.CodigoDiagnosticoRelacionado3)
        break

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD213297" CentroA="01" CodigoDiagnosticoRelacionado1="Z359" CodigoDiagnosticoRelacionado2="Z988" CodigoDiagnosticoRelacionado3="" Diagnostico="O479" IdPaciente="162505" Modulo="01" Padre="AD213297" PlanAdm="100" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  1-&amp;amp;gt; ASMA
MENARQUIA: 10 A&amp;#209;OS.
VIDA MARITAL: 14 A&amp;#209;OS.
VIDA OBSTETRICA: 14 A&amp;#209;OS.
G2 P1 C0 A0  
CICLOS: 7X30
FUM: NO RECUERDA
EG X FUM: 
COMPA&amp;#209;EROS SEXUALES: 2
GRUPO SANGUINEO: NO SABE 
PLANIFICACI&amp;#211;N FAMILIAR: NIEGA
CONTROLES PRENATALES: 0

ECOGRAFIA 21/11/16: EMBARAZO DE 24.6 SEMANAS PARA HOY 38.4 
ECOGRAFIA 16/02/17: EMBARAZO DE 37.4 SEMANAS PARA HOY 38.6

PARACLINICOS:9/11/2016
VDRL: NO REACTIVO 
HB:10.3   HTO:31   
ANTECEDENTE: A

In [26]:
alta = 0
noAlta = 0
for k, b in backgroundByCase.items():
    if len(b) == 0:
        continue
    if all(b.AltaMedica):
        alta += 1
    else: 
        noAlta += 1
print(alta, noAlta)

0 2462


In [ ]:
def caseToBackground(case, backgroundByCase):
    p = backgroundByCase[case]
    if len(p) != 0:  
        text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    return text

In [126]:
import unicodedata
def remove_diacritics(text):
    """
    Returns a string with all diacritics (aka non-spacing marks) removed.
    For example "Héllô" will become "Hello".
    Useful for comparing strings in an accent-insensitive fashion.
    """
    normalized = unicodedata.normalize("NFKD", text)
    return "".join(c for c in normalized if unicodedata.category(c) != "Mn")


case, _, _ = list(interventionToCase.values())[423]
print(case)
p = backgroundByCase[case]
if len(p) != 0:  
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    textClean = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    print(textClean)
else:
    print('Error')

AD337645
- PERSONALES:
  7-&gt; MENARQUIA: 9ANOS 


VIDA MARITAL: 17 OS
VIDA OBSTETRICA: 18NOS
G3P2 
FUM: NO CONFIABLE 
PLANIFICACION: NIEGA 
GRUPO SANGUINEO: O+ 
CONTROLES PRENATALES: 0 

PARACLINICOS NO TRAE
ECOGRAFIA: NO TRAE
- FAMILIARES:
  NIEGA
- ALERGICOS:
  
- QUIRURGICOS:


/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


# Background

Check from the yes / no form. 

The ones that have, try to get the most common. The rest note for manual

In [ ]:
prettyPrintXML(registros[(registros.Caso == case) & registros.Asunto.str.contains('Ingreso de Urgencias')].iloc[0].RegistroXML)

# Paraclinics and background familiar

- From the antecedentes? There are many without anything in the venezuelans. Are hey applied during labour?
- Much variability -> need to clean manually, but it might be too costly

See what are the tests done (in orders, and the medication)

In [123]:
# Maybe I should start tokenizing
cleanWhites = re.compile("[^\S\n]+")
#cleanWhites.sub(' ', text)

def cleanString(text, removeChars = '-:,;', removeWords = []):
    for c in removeChars:
        text = re.sub('(?<![0-9])\%s' %c, ' ',  text)
        text = re.sub('\%s(?![0-9])' %c, ' ',  text)

    text = cleanWhites.sub(' ', text)
    for w in removeWords:
        text = text.replace(' ' + w + ' ', ' ')
    return text.strip()

In [ ]:
#Parse paraclinics
#VIH
#VDRL -
# PRUEBA RAPIDA DE SIFILIS
dict = {}
for (case, _, _) in interventionToCase.values():
    p = backgroundByCase[case]
    if len(p) == 0:
        continue
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    if len(r) == 0:
        continue
       # text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    dict[case] = {'VIH': 'VIH' in text, 'VDRL' : 'VDRL' in text, 'PARACLINICOS' :  'PARACLINICOS' in text, 'SIF' : 'PRUEBA RAPIDA DE SIFILIS' in text}

In [ ]:
paraclinics = pandas.DataFrame.from_dict(dict, orient = 'index')
paraclinics.sum(), len(paraclinics)

In [ ]:
def parseAntecedentes(t):
    t = t.upper()
    antecedentes = []
    negative = ['NO', 'NIEG', 'SIN DATOS', 'NEGATIVO', 'NO REFIERE', 'SIN']
    if 'HIPERTEN' in t or 'HTA' in t:
        antecedentes.append('HTA')
    if any([n in t for n in negative]):
        antecedentes.append('None')
    if 'DIAB' in t or 'DM' in t:
        antecedentes.append('Diabetes')
    if 'ASMA' in t:
        antecedentes.append('Asma')
    if 'CARDIO' in t:
        antecedentes.append('Cardo')
    if 'PREECLAMPSIA' in t:
        antecedentes.append('Preclampsia')

    return antecedentes

In [ ]:
j  = 0
noFam, noPara = 0, 0
fam = {}
para = {}
for k, r in paraclinics.iterrows(): 
    text = caseToBackground(k, backgroundByCase)
    paraclinicos = re.findall('PARACLINICOS(.*)FAMILIARES', text, re.DOTALL)
    familiares = re.findall('FAMILIARES(.*)ALERGICOS', text, re.DOTALL)
    if not paraclinicos:
        noPara += 1
    else: 
        
    else:
        para[k] = cleanString(paraclinicos[0])
    if not familiares:
        noFam += 1
    else:
        print(cleanString(familiares[0]))
        fam[k] = parseAntecedentes(cleanString(familiares[0]))
print(noFam, noPara, len(paraclinics))

In [ ]:
for c in any.values():
    print(c)

# Echographies:

- There is a lot of variability that makes it difficult to get all of them.
- In most cases I should be able to get at least one.
- Do-able: GA. If there is something more, do it manually

In [ ]:
noEchos = ['no trae ecografia', 'ni ecografia']

In [ ]:

embarazo = ['embarazo de', 'emb de', 'embarazo', 'emb', 'QUE REPORTA EMBARAZO DE']
query = 'Ecografi[a]?[s]?' + sep +  '(' + sep +  date + sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(%s)' % floatParse  + blanks + semanas \
+ blanks  + '(:?'+ paraHoy + floatParse +  ')?' + ')+'
re.findall(query, text, re.IGNORECASE)

In [ ]:
blanks = '\s*'
paraHoyPossibilities = ['Para hoy', 'hoy', 'a hoy', 'para el dia de hoy']
paraHoy = '(?:%s)' % '|'.join(paraHoyPossibilities)
semanas = '(?:%s)' % '|'.join(['semanas', 'sem', 'sems'])

echoLine =  sep +  date +  sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(' + '(%s)' % floatParse  + blanks + semanas  + '[,]?'\
+ blanks + paraHoy + blanks + floatParse + blanks + '(:?%s)?' % semanas + ')?'

queryEchos = '(Eco[g]?[r]?[a]?[f]?[i]?[a]?[s]?' + sep +  '(' + echoLine  + ')+)'
def searchEchos(text):
    for t in re.findall(query, text, re.IGNORECASE):
        print(t)
    #TODO: afterwards parse all echographies from the result

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [ ]:
backgroundByCase['AD301211'] = getBackgroundRegister(case, registersByPatient.get_group(pId))

In [ ]:
registros[registros.NumeroHistoria == 216835]

In [ ]:
#TODO: check that operation and diagnosis are compatible.

In [ ]:
getprocID(i.XmlDescripcion)
for k,c in interventionToCase[k].iterrows():
    prettyPrintXML(c.RegistroXML)

# Patient information

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

# Classify intervention

In [31]:
for c, v in interventionToCase.items():
    interventionID = getProcFromXMLString(procedimientos.loc[c].XmlDescripcion)
    break

17640 ('AD152124', 139326, 550963)


# New born information and birth

- New born anthropometrics
- Birth details (quirurgic)
- Life  /dead, and also of the mum
- Whether there was a single one, or many
- C- section

In [59]:
classificationProcedures = {'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}

In [212]:
def findDesgarros(text):
    text = text.replace(' de ', ' ').replace(' se ', ' ')
    
    removeWords = ['lateral', 'izquierda', 'derecha', 'superior', 'inferior', 'medial']
    for w in removeWords:
        text = text.replace(' %s ' % w, ' ')

    #Perdida de sangre
    bloodLost = re.findall('perdida estimada sangre(?::)? ([0-9]+) (?:cc|ml)', text)
    ver = '(?:eviden[a-z]*|observ[a-z]*|vis[a-z]*|encont[a-z]*|presen[a-z]*)'   #diferentes manaeras de escribir ver
    negative = ['(?:sin|no) (?:%s )?desgar' % ver]
    positive = 'desgar[a-z]* (?:[a-z]* |(:?pared )?vag[a-z]* )?(?:sangr[a-z]* |no sangrant[a-z]* )?grado (i|ii|ii|1|2|3)'
    positiveUnidentified = '(?:%s )?desgar' % ver

    if re.findall('(%s)' % '|'.join(negative), text) or ('desgarro' not in text): #and 'sin complicaciones' in text):
        desgarro = 'no'
    elif re.findall(positive, text):
        desgarro = 'yes'
    elif re.findall(positiveUnidentified, text):
        desgarro = 'yes-NoGrade'
    else:
        desgarro = text
    return bloodLost, desgarro

In [215]:
wrongParse = 0
for c,v in interventionToCase.items():
    xmlDesc = procedimientos.loc[c].XmlDescripcion
    idProc = getProcFromXMLString(procedimientos.loc[c].XmlDescripcion)
    if classificationProcedures[idProc] == 'p':
        et = ET.fromstring(xmlDesc)
        r =  findDesgarros(remove_diacritics(cleanString(et.find('detalle/procedimientos/procedimiento/descripcion').text.lower())))
        wrongParse += r[1] not in ['yes', 'no', 'yes-NoGrade']
        print(r)

(['250'], 'no')
(['250'], 'no')
(['200'], 'no')
(['150'], 'no')
(['300'], 'yes')
(['200'], 'no')
([], 'no')
(['300'], 'no')
(['200'], 'yes')
(['150'], 'yes')
(['250'], 'no')
(['200'], 'no')
(['200'], 'no')
(['200'], 'yes')
(['350'], 'no')
(['200'], 'no')
(['150'], 'no')
(['150'], 'no')
([], 'no')
(['200'], 'no')
(['200'], 'no')
(['150'], 'no')
(['200'], 'yes-NoGrade')
([], 'no')
([], 'no')
(['150'], 'yes-NoGrade')
([], 'no')
([], 'yes')
(['200'], 'no')
(['200'], 'yes')
([], 'no')
([], 'no')
([], 'no')
(['200'], 'no')
([], 'no')
(['200'], 'yes')
([], 'no')
([], 'yes')
(['200'], 'yes-NoGrade')
([], 'no')
(['150'], 'no')
(['200'], 'no')
([], 'no')
([], 'no')
(['100'], 'no')
([], 'yes')
(['200'], 'no')
([], 'no')
([], 'no')
(['150'], 'no')
(['300'], 'no')
(['150'], 'no')
([], 'no')
(['150'], 'yes')
(['200'], 'no')
([], 'no')
([], 'no')
(['80'], 'no')
(['100'], 'no')
([], 'no')
(['300'], 'yes')
(['250'], 'no')
([], 'no')
(['200'], 'no')
([], 'yes-NoGrade')
(['100'], 'no')
([], 'no')
(['100'

In [216]:
wrongParse, len(interventionToCase.items())

(0, 2572)

# Abortion

In [40]:
getProcFromXMLString(procedimientos.loc[17640].XmlDescripcion)

'H3100'

In [39]:
procedimientos.loc[17640]

FechaRegistro                                 2016-03-03 13:43:27.833
FechaDescripcion                                  2016-03-03 01:41:00
ActoQuirurgico                                                CP23726
IdPaciente                                                     139326
Usuario                          CEACBACD-0BBE-4709-9803-4AD990D5A45E
XmlDescripcion      <descripcionQuirurgica><cabecera><quirofano>04...
Xslt                <?xml version="1.0" encoding="utf-8"?><xsl:sty...
Name: 17640, dtype: object